In [60]:
import spotipy
import json
import webbrowser
import urllib.request
import overpass
import spotipy.util as util
import geohash
import pygeohash as pgh

In [61]:
with open("geocode_keys.txt", "r") as geokey_file:
    geo_key = geokey_file.read()

In [62]:
address = "15 Elm St, Totowa, NJ, 07512, US"
nospace_address = (
    address
        .replace(",", "")
        .replace(" ", "+")
)

# print(nospace_address)

map_url = f"https://geocode.maps.co/search?q={nospace_address}&api_key={geo_key}"

map_request = urllib.request.Request(map_url)
map_response = urllib.request.urlopen(map_request)

map_data = json.loads(map_response.read())
lat = map_data[0]['lat']
lon = map_data[0]['lon']
print(lat, lon)

40.923232 -74.202799


In [63]:
test_hash = pgh.encode(40.923232, -74.202799, precision=5)
print(test_hash)

dr70z


In [64]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

with open("genreIDs.json", "r") as genreID_file:
    genre_list = json.loads(genreID_file.read())

print(genre_list['rap'])

KnvZfZ7vAv1


In [65]:
geohash = "gcpuv"
unit = "km"
radius = "50"
genre = genre_list['country']
url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&radius={radius}&genreId={genre}&geoPoint={test_hash}&classificationName=music&apikey={ticketmaster_key}"

tm_request = urllib.request.Request(url)
tm_response = urllib.request.urlopen(tm_request)

full_event_listings = json.loads(tm_response.read())
event_names = full_event_listings['_embedded']['events']
print(event_names)


[{'name': 'Down South: Ultimate Country Dance Party', 'type': 'event', 'id': '1Ae0ZbfGkIxhvMf', 'test': False, 'url': 'https://www.ticketweb.com/event/down-south-ultimate-country-dance-dingbatz-tickets/13661714?REFERRAL_ID=tmfeed', 'locale': 'en-us', 'images': [{'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': True}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_RETINA_PORTRAIT_16_9.jpg', 'width': 640, 'height': 360, 'fallback': True}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': True}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_RETINA_PORTRAIT_3_2.jpg', 'width': 640, 'height': 427, 'fallback': True}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/

In [66]:
artist_names = [artist['name'] for artist in event_names]
artist_names_no_repeats = list(dict.fromkeys(artist_names))

print(artist_names_no_repeats)

['Down South: Ultimate Country Dance Party', 'Kelsea Ballerini - Live On Tour', 'Cody Johnson - Leather Deluxe Tour', 'Blake Shelton: Friends & Heroes 2025', 'Kacey Musgraves - Deeper Well World Tour', 'Megan Moroney - Am I Okay? Tour', "On A Winter's Night ft. Cliff Eberhardt, John Gorka, Lucy Kaplansky & Patty Larkin", 'Folsom Prison Experience', "STEVE EARLE: 10TH ANNUAL JOHN HENRY'S FRIEND BENEFIT", 'Brett Eldredge - GLOW: Welcome to the Family Tour', 'Trapper Schoepp, Hannah Aldridge, Kellen Asebroek', 'Twisted Pine [Love Your Mind Release Tour] & Damn Tall Buildings', 'Dylan Gossett - The Back 40 Tour', 'Ryan Adams', 'Mac McAnally']


In [67]:
cred = "spotify_keys.json"
with open(cred, "r") as spotify_key_file:
    api_tokens = json.load(spotify_key_file)

spotify_key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
sp = spotipy.Spotify(auth=token)

playlist_songs = []

for artist in artist_names_no_repeats:
    returned_song = sp.search(q=artist, type="track", limit=1)

    if returned_song['tracks']['items'][0]['uri'] not in playlist_songs:
        playlist_songs.append(returned_song['tracks']['items'][0]['uri'])

# print(playlist_songs)

empty_playlist = sp.user_playlist_create(user=username, name=f"What's on in {address.upper()}", public=True, description=f"songs from {genre} artists who have played live within a {radius+unit} radius of {address.upper()}")
full_playlist = sp.user_playlist_add_tracks(username, empty_playlist['id'], playlist_songs)

In [69]:
webbrowser.open(empty_playlist['external_urls']['spotify'])

True